In [31]:
#设置url
#比特币交易网 https://www.btctrade.com
#网站api页面 https://www.btctrade.com/api.help.html#open
url_btctrade = "api.btctrade.com"
#获取行情 (GET) high: 最高价,low: 最低价,buy: 买一价,sell: 卖一价,last: 最新成交价,vol: 成交量(最近的24小时),time: 返回数据时服务器时间
btctrade_ticker = "http://api.btctrade.com/api/ticker?coin=eth"
#深度行情 (GET) asks: 卖方深度[价格, 委单量]，价格从高到低排序,bids: 买方深度[价格, 委单量]，价格从高到低排序
btctrade_depth = "http://api.btctrade.com/api/depth?coin=eth"
#成交记录 (GET) date: 成交时间,price: 交易价格,amount: 交易数量,tid: 交易ID,type: 交易类型
btctrade_trades = "http://api.btctrade.com/api/trades?coin=eth"



In [32]:
#创建一个文件夹 coindata 用来存数据
import os
if os.path.exists('./coindata') == False:
    os.mkdir("./coindata")
    print('mkdir coindata. ')


A


In [33]:
#定义相应的写到文件的方法
def ticker_write(datetime,ticker):
    string = "%s,%s,%s,%s,%s,%s,%s\r\n" % (ticker['high'],ticker['low'],ticker['buy'],ticker['sell'],ticker['last'],ticker['vol'],ticker['time'])
    fp = open("./coindata1/ticker_data.csv",'a')
    fp.write(datetime+string)
    fp.close()
    
def depth_write(datetime,depth):    
    index = 0
    bid_price_list = ['']*len(depth['bids'])
    bid_vol_list = ['']*len(depth['bids'])
    ask_price_lit = ['']*len(depth['asks'])
    ask_vol_lit = ['']*len(depth['asks'])    
    for i in depth['bids']:
        bid_price_list[index] = i[0]
        bid_vol_list[index] = i[1]
        index = index + 1
    index = 0    
    for i in depth['asks']:
        ask_price_lit[index] = i[0]
        ask_vol_lit[index] = i[1]
        index = index + 1    
    fp = open("./coindata1/depth_data.csv",'a')
    fp.write(datetime+','.join(bid_price_list)+','+','.join(bid_vol_list)+','+','.join(ask_price_lit)+','+','.join(ask_vol_lit)+'\r\n')
    fp.close()
    
def trades_write(trades_tmp,trades):  
    fp = open("./coindata1/trades_data.csv",'a')
    #只要trades中的数据，不在trades_tmp中，就写到文件中
    for trade in trades:
        if (trade in trades_tmp) == False:
            timeArray = time.localtime(float(trade['date']))
            otherStyleTime = time.strftime("%Y%m%d,%H:%M:%S,", timeArray)
            string = otherStyleTime+"%s,%s,%s,%s,%s\r\n"% (trade['date'],trade['tid'],trade['price'],trade['type'],trade['amount'])
            fp.write(string)
    fp.close()

In [ ]:
#测试深度行情 多少秒更新一次比较好
import httplib
import time
import json

headers = {"Content-type": "application/x-www-form-urlencoded",
           "Accept": "application/json","Content-type":"application/xml; charset=utf=8"}  

ticker_json_tmp = ''
depth_json_tmp = ''
trades_json_tmp = ''
trades_tmp = ['']

#这个网上的数据显示 script设置的200毫秒更新一次,待需要测试0.1秒
sleep_sec = 0.2
#时间变量，放在外部
ticker_time = ''
otherStyleTime = ''


while(True):
    time.sleep(sleep_sec)
    
    #try catch，出现异常不会停止，在循环中继续获取数据
    try:
        #timeout 设置为 None，要不会有 error 10060的错，http request的问题，如果设置1秒回超时
        conn = httplib.HTTPConnection(url_btctrade,timeout=None)
        #首先获取行情确定时间
        conn.request("GET",btctrade_ticker,'',headers)
        ticker_response = conn.getresponse()
        #conn.request("GET",btctrade_ticker,"",headers)

        if ticker_response.status == 200:
            ticker_json = ticker_response.read()
            #如果从新获取的数据相比上一次获取有更新
            if ticker_json_tmp != ticker_json:
                #在这里可以需要存数据 ticker_json
                #print 1
                #确定时间
                ticker = json.loads(ticker_json) 
                ticker_time = time.localtime(ticker['time'])
                otherStyleTime = time.strftime("%Y%m%d,%H:%M:%S,", ticker_time)

                ticker_write(otherStyleTime,ticker)
                #copy到tmp
                ticker_json_tmp = ticker_json

        #继续获取深度行情
        conn.request("GET",btctrade_depth,'',headers)
        depth_response = conn.getresponse()

        if depth_response.status == 200:
            depth_json = depth_response.read()
            if depth_json_tmp != depth_json:
                #这里存数据 depth_json
                #print 2
                depth = json.loads(depth_json)
                depth_write(otherStyleTime,depth)
                #copy到tmp
                depth_json_tmp = depth_json

        #继续获取成交记录
        conn.request("GET",btctrade_trades,'',headers)
        trades_response = conn.getresponse()

        if trades_response.status == 200:
            trades_json = trades_response.read()

            if trades_json_tmp != trades_json:
                #这里存数据 trades_json
                #print 3

                trades = json.loads(trades_json)
                trades_write(trades_tmp,trades)
                trades_tmp = trades
                #copy到tmp
                trades_json_tmp = trades_json

        #每次循环开始都是创建新连接，然后结束关闭连接
        conn.close()
    except Exception, e:
        print "Exception: ",e
    finally:
        if conn:
            conn.close()

Exception:  [Errno 10060] 
Exception:  [Errno 10060] 
Exception:  [Errno 10060] 
Exception:  [Errno 10060] 
Exception:  [Errno 11004] getaddrinfo failed
Exception:  [Errno 11004] getaddrinfo failed
Exception:  [Errno 11004] getaddrinfo failed
Exception:  [Errno 11004] getaddrinfo failed
Exception:  [Errno 11004] getaddrinfo failed
Exception:  [Errno 11004] getaddrinfo failed
Exception:  [Errno 11004] getaddrinfo failed
Exception:  [Errno 11004] getaddrinfo failed
Exception:  [Errno 11004] getaddrinfo failed
Exception:  [Errno 11004] getaddrinfo failed
Exception:  [Errno 11004] getaddrinfo failed
Exception:  [Errno 11004] getaddrinfo failed
Exception:  [Errno 11004] getaddrinfo failed
Exception:  [Errno 11004] getaddrinfo failed
Exception:  [Errno 11004] getaddrinfo failed
Exception:  [Errno 11004] getaddrinfo failed
Exception:  [Errno 11004] getaddrinfo failed
Exception:  [Errno 11004] getaddrinfo failed
Exception:  [Errno 11004] getaddrinfo failed
Exception:  [Errno 11004] getaddrinfo

In [ ]:
#下面都是测试代码,调试程序写的。

In [169]:
#python调用restapi

#创建一个文件夹 coindata 用来存数据
import os
if os.path.exists('./coindata') == False:
    os.mkdir("./coindata")
    
#设置http headers
import httplib
headers = {"Content-type": "application/x-www-form-urlencoded",
           "Accept": "application/json","Content-type":"application/xml; charset=utf=8"}  

conn = httplib.HTTPConnection(url_btctrade)
conn.request("GET",btctrade_ticker,"",headers)
response = conn.getresponse()
tickerjson = ''
#打印连接信息
print "Return code:", response.status, " reason:", response.reason 
print response.status 

import json
if response.status == 200:
    tickerjson = response.read()
    ticker = json.loads(tickerjson)
    import time 
    print time.ctime(ticker['time'])
    print "%s,%s,%s,%s,%s,%s,%s" % (ticker['high'],ticker['low'],ticker['buy'],ticker['sell'],ticker['last'],ticker['vol'],ticker['time'])
    
#while(True):
    #线程sleep
    #time.sleep(1)
    #print 1
    
#判断两个httpresponse的body是否一样
time.sleep(4)
conn2 = httplib.HTTPConnection(url_btctrade)
conn2.request("GET",btctrade_ticker,"",headers)
response2 = conn2.getresponse()
tickerjson2 = ''

if response2.status == 200:
    tickerjson2 = response2.read()
    ticker = json.loads(tickerjson2)
    import time 
    print time.ctime(ticker['time'])
    print "%s,%s,%s,%s,%s,%s,%s" % (ticker['high'],ticker['low'],ticker['buy'],ticker['sell'],ticker['last'],ticker['vol'],ticker['time'])
print tickerjson 
print tickerjson2
print tickerjson == tickerjson2 #判断是否相等直接就用==就行了

Return code: 200  reason: OK
200
Thu Mar 02 16:04:40 2017
103.7,92.95,101.71,102.08,101.8,49391.08,1488441880
Thu Mar 02 16:04:49 2017
103.7,92.95,101.71,102.08,101.8,49391.08,1488441889
{"high":103.7,"low":92.95,"buy":101.71,"sell":102.08,"last":101.8,"vol":"49391.08","time":1488441880}

{"high":103.7,"low":92.95,"buy":101.71,"sell":102.08,"last":101.8,"vol":"49391.08","time":1488441889}

False


In [300]:
import time

def ticker_write(datetime,ticker):
    string = "%s,%s,%s,%s,%s,%s,%s\r\n" % (ticker['high'],ticker['low'],ticker['buy'],ticker['sell'],ticker['last'],ticker['vol'],ticker['time'])
    fp = open("./coindata/testa.txt",'a')
    fp.write(datetime+string)
    fp.close()

conn2 = httplib.HTTPConnection(url_btctrade)
conn2.request("GET",btctrade_ticker,"",headers)
response2 = conn2.getresponse()
tickerjson2 = ''

if response2.status == 200:
    tickerjson2 = response2.read()
    ticker = json.loads(tickerjson2)
    
    timeArray = time.localtime(ticker['time'])
    otherStyleTime = time.strftime("%Y%m%d,%H:%M:%S,", timeArray)
    print otherStyleTime
    
    ticker_write(otherStyleTime,ticker)

#存深度行情的算法
def depth_write_test(datetime,depth):
    fp = open("./coindata/testb.txt",'a')
    fp.write('bbb\r\n')
    #print depth
    print depth['bids']
    print depth['asks']
    print type(depth['bids'])
    print len(depth['bids'])
    print depth['bids'][0]
    print type(depth['bids'][0])
    print len(depth['bids'][0])
    print depth['bids'][0][0]
    listbid = ['']*len(depth['bids'])
    print ','.join(depth['bids'][0])
    listbid[0] = ','.join(depth['bids'][0])
    listbid[1] = ','.join(depth['bids'][1])
    print listbid
    
    index = 0
    bid_price_list = ['']*len(depth['bids'])
    bid_vol_list = ['']*len(depth['bids'])
    ask_price_lit = ['']*len(depth['asks'])
    ask_vol_lit = ['']*len(depth['asks'])
    
    for i in depth['bids']:
        bid_price_list[index] = i[0]
        bid_vol_list[index] = i[1]
        index = index + 1
    index = 0    
    for i in depth['asks']:
        ask_price_lit[index] = i[0]
        ask_vol_lit[index] = i[1]
        index = index + 1    
        
    print ','.join(bid_price_list)
    print ','.join(bid_vol_list)
    print ','.join(ask_price_lit)
    print ','.join(ask_vol_lit)
    fp.close()
def depth_write(datetime,depth):    
    index = 0
    bid_price_list = ['']*len(depth['bids'])
    bid_vol_list = ['']*len(depth['bids'])
    ask_price_lit = ['']*len(depth['asks'])
    ask_vol_lit = ['']*len(depth['asks'])
    
    for i in depth['bids']:
        bid_price_list[index] = i[0]
        bid_vol_list[index] = i[1]
        index = index + 1
    index = 0    
    for i in depth['asks']:
        ask_price_lit[index] = i[0]
        ask_vol_lit[index] = i[1]
        index = index + 1    
    fp = open("./coindata/testbb.txt",'a')
    fp.write(datetime+','.join(bid_price_list)+','+','.join(bid_vol_list)+','+','.join(ask_price_lit)+','+','.join(ask_vol_lit)+'\r\n')
    fp.close()
    
    
conn2 = httplib.HTTPConnection(url_btctrade)
conn2.request("GET",btctrade_depth,"",headers)
response2 = conn2.getresponse()
tickerjson2 = ''

if response2.status == 200:
    tickerjson2 = response2.read()
    ticker = json.loads(tickerjson2)
    
    print otherStyleTime
    
    depth_write(otherStyleTime,ticker)

def trades_write(trades_tmp,trades):  
    print trades
    print type(trades)
    print len(trades)
    lastone = trades[-1]
    print lastone
    print (lastone in trades) == False
    
    fp = open("./coindata/testc.txt",'a')
    #只要trades中的数据，不在trades_tmp中，就写到文件中
    for trade in trades:
        if (trade in trades_tmp) == False:
            timeArray = time.localtime(float(trade['date']))
            otherStyleTime = time.strftime("%Y%m%d,%H:%M:%S,", timeArray)
            string = otherStyleTime+"%s,%s,%s,%s,%s\r\n"% (trade['date'],trade['tid'],trade['price'],trade['type'],trade['amount'])
            print string
            fp.write(string)
    fp.close()
    
    
conn2 = httplib.HTTPConnection(url_btctrade)
conn2.request("GET",btctrade_trades,"",headers)
response2 = conn2.getresponse()
tickerjson2 = ''

if response2.status == 200:
    tickerjson2 = response2.read()
    ticker = json.loads(tickerjson2)
    
    print otherStyleTime
    trades_write([''],ticker)
    


20170303,14:53:40,
20170303,14:53:40,
20170303,14:53:40,
[{u'date': u'1488522617', u'tid': u'30028230', u'price': 123.41, u'type': u'buy', u'amount': 27.57}, {u'date': u'1488522649', u'tid': u'30028233', u'price': 123.6, u'type': u'buy', u'amount': 39.492}, {u'date': u'1488522650', u'tid': u'30028236', u'price': 123.6, u'type': u'sell', u'amount': 0.5}, {u'date': u'1488522745', u'tid': u'30028251', u'price': 123.98, u'type': u'buy', u'amount': 2.277}, {u'date': u'1488522767', u'tid': u'30028253', u'price': 123.98, u'type': u'buy', u'amount': 0.787}, {u'date': u'1488522821', u'tid': u'30028256', u'price': 124, u'type': u'buy', u'amount': 9.814}, {u'date': u'1488522922', u'tid': u'30028259', u'price': 124.01, u'type': u'sell', u'amount': 0.29}, {u'date': u'1488522922', u'tid': u'30028262', u'price': 124.01, u'type': u'sell', u'amount': 0.67}, {u'date': u'1488523150', u'tid': u'30028274', u'price': 124.2, u'type': u'buy', u'amount': 0.186}, {u'date': u'1488523172', u'tid': u'30028277', u'